<a href="https://colab.research.google.com/github/Adarejohnson/Fashion-dataset-dashboard/blob/main/Intrum_casestudy_pythoncode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd # Loading pandas library
from datetime import datetime, timedelta # Loading datetime library
casestudy_df = pd.read_csv('/content/sample_data/CaseStudy Data.csv') # Loading casestudy as dataframe
casestudy_df.head() # print dataframe


,CustomerReference,TotalToPay,FirstPayment,FirstPaymentDate,NumberOfPayments,InstalmentAmount,Frequency,FrequencyType,FrequencyNumber
0,XXXXXXXX1,430.86,430.86,2020-07-08,0,0.0,single,D,0
1,XXXXXXXX2,143.62,143.62,2020-07-16,0,0.0,single,D,0
2,XXXXXXXX3,200.56,200.56,2020-07-15,0,0.0,single,D,0
3,XXXXXXXX4,89.60,89.60,2020-08-29,0,0.0,single,D,0
4,XXXXXXXX5,181.23,181.23,2020-10-19,0,0.0,single,D,0


Data Pre-processing, check for null values and duplicates

In [2]:
casestudy_df.isnull().sum() # check for null values

CustomerReference    0
TotalToPay           0
FirstPayment         0
FirstPaymentDate     0
NumberOfPayments     0
InstalmentAmount     0
Frequency            0
FrequencyType        0
FrequencyNumber      0
dtype: int64

In [3]:
casestudy_df.duplicated().sum() # check for duplicates

0

In [4]:
casestudy_df.info() # check data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687 entries, 0 to 686
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerReference  687 non-null    object 
 1   TotalToPay         687 non-null    float64
 2   FirstPayment       687 non-null    float64
 3   FirstPaymentDate   687 non-null    object 
 4   NumberOfPayments   687 non-null    int64  
 5   InstalmentAmount   687 non-null    float64
 6   Frequency          687 non-null    object 
 7   FrequencyType      687 non-null    object 
 8   FrequencyNumber    687 non-null    int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 48.4+ KB


In [10]:
print(casestudy_df.apply(lambda Frequency: Frequency.unique()))

CustomerReference    [XXXXXXXX1, XXXXXXXX2, XXXXXXXX3, XXXXXXXX4, X...
TotalToPay           [430.86, 143.62, 200.56, 89.6, 181.23, 1036.51...
FirstPayment         [430.86, 143.62, 200.56, 89.6, 181.23, 86.87, ...
FirstPaymentDate     [2020-07-08, 2020-07-16, 2020-07-15, 2020-08-2...
NumberOfPayments     [0, 12, 16, 104, 195, 5, 6, 7, 8, 9, 10, 3, 18...
InstalmentAmount     [0.0, 86.87, 45.7, 100.0, 300.52, 321.08, 377....
Frequency                                    [single, monthly, weekly]
FrequencyType                                                   [D, M]
FrequencyNumber                                              [0, 1, 7]
dtype: object


In [11]:
def calculate_payments(row): # Creating function to calculate planned payments
    start_date = datetime.strptime(row['FirstPaymentDate'], '%Y-%m-%d')
    payments = []
    total_to_pay = row['TotalToPay']
    first_payment = row['FirstPayment']
    installment_payment = row['InstalmentAmount']
    frequency_number = row['FrequencyNumber']
    frequency_type = row['FrequencyType']

    if frequency_type == 'D': # converting the frequency to days
        frequency = timedelta(days=frequency_number)
    elif frequency_type == 'M':
        frequency = timedelta(days=frequency_number * 30) # Using 30days cycle
    else:
        raise ValueError(f"Unsupported FrequencyType: {frequency_type}")
       # Apply first payment
    payments.append({
        'PaymentDate': start_date.strftime('%Y-%m-%d'),
        'PaymentAmount': first_payment
    })
    total_to_pay -= first_payment
    current_date = start_date + frequency

    # Calculate subsequent payments
    while total_to_pay > 0:
        if total_to_pay < installment_payment:
            installment_payment = total_to_pay
        payments.append({
            'PaymentDate': current_date.strftime('%Y-%m-%d'),
            'PaymentAmount': installment_payment
        })
        total_to_pay -= installment_payment
        current_date += frequency

    return payments



In [12]:
# Apply the function to each row
casestudy_df['PlannedPayments'] = casestudy_df.apply(calculate_payments, axis=1)

# Extract the last payment date and amount
def get_last_payment(payments):
    last_payment = payments[-1]
    return pd.Series([last_payment['PaymentDate'], last_payment['PaymentAmount']])

casestudy_df[['LastPaymentDate', 'LastPaymentAmount']] = casestudy_df['PlannedPayments'].apply(lambda x: get_last_payment(x))

In [13]:
# Example output
print(casestudy_df[['CustomerReference', 'PlannedPayments', 'LastPaymentDate', 'LastPaymentAmount']])

    CustomerReference                                    PlannedPayments  \
0           XXXXXXXX1  [{'PaymentDate': '2020-07-08', 'PaymentAmount'...   
1           XXXXXXXX2  [{'PaymentDate': '2020-07-16', 'PaymentAmount'...   
2           XXXXXXXX3  [{'PaymentDate': '2020-07-15', 'PaymentAmount'...   
3           XXXXXXXX4  [{'PaymentDate': '2020-08-29', 'PaymentAmount'...   
4           XXXXXXXX5  [{'PaymentDate': '2020-10-19', 'PaymentAmount'...   
..                ...                                                ...   
682         XXXXXX683  [{'PaymentDate': '2022-11-10', 'PaymentAmount'...   
683         XXXXXX684  [{'PaymentDate': '2022-11-30', 'PaymentAmount'...   
684         XXXXXX685  [{'PaymentDate': '2022-11-09', 'PaymentAmount'...   
685         XXXXXX686  [{'PaymentDate': '2022-11-10', 'PaymentAmount'...   
686         XXXXXX687  [{'PaymentDate': '2022-11-15', 'PaymentAmount'...   

    LastPaymentDate  LastPaymentAmount  
0        2020-07-08             430.86  
1    

In [14]:
# Save the results to an Excel file
casestudy_df.to_excel('/content/sample_data/arrangement_payments.xlsx', index=False)